# Reciprocal Rank Fusion (RRF) + Multi-Query Retrieval Demo

This notebook demonstrates a retrieval fusion pattern that combines multiple retrieval results using Reciprocal Rank Fusion (RRF) to produce a consolidated ranking of candidate documents. The goal is to improve retrieval robustness by aggregating results from multiple query variants generated by an LLM.

## What this notebook contains
- Generating multiple query variants from a single user question using a Chat LLM.
- Retrieving documents for each variant using a vector store (Chroma) and OpenAI embeddings.
- Applying Reciprocal Rank Fusion (RRF) to merge ranked result lists.
- Selecting top candidates and using them with a RAG prompt to produce a final answer.

## References
https://towardsdatascience.com/how-to-make-your-llm-more-accurate-with-rag-fine-tuning/

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  
from langchain_community.document_loaders import WebBaseLoader  
from langchain_community.vectorstores import Chroma  
from langchain_core.output_parsers import StrOutputParser  
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain.prompts import ChatPromptTemplate
from langchain.load import dumps, loads
from operator import itemgetter
import yaml
import bs4  
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Get the current working directory
cwd = os.getcwd()

# Build the path to config.yaml
config_path = os.path.join(cwd, '..', 'configs', 'config.yaml')

# Normalize the path
config_path = os.path.abspath(config_path)

# Load credential from config file
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Set environment variables
os.environ['LANGCHAIN_API_KEY'] = config['API']['LANGCHAIN']
os.environ['OPENAI_API_KEY'] = config['API']['OPENAI']

In [3]:
# Create a loader that fetches and parses the target web page
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),  # tuple of URLs to load
    bs_kwargs=dict(  # pass BeautifulSoup-specific kwargs to limit parsing
        parse_only=bs4.SoupStrainer(  # only parse these parts of the page to reduce noise
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Fetch and return a list of Document objects
docs = loader.load()  

# Split long documents into smaller overlapping chunks suitable for embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)  # list of smaller Document chunks

# Create embeddings and store them in a vector DB (Chroma)
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())  # uses OpenAI embeddings under the hood

# Create a retriever to fetch relevant docs (return the top 5 results)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [4]:
# Ask an LLM to rephrase the question into multiple variants (generate mult-queries)
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to:  {question} \n
Output (5 queries):
"""

# Build the prompt object
prompt_perspectives = ChatPromptTemplate.from_template(template)

# Pipeline: prompt -> chat model -> parse -> split into separate queries
generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))  # split on newlines into a list
)

In [5]:
# Example input question
question = "What is task decomposition for LLM agents?"

# Generate multiple query variants
result = generate_queries.invoke({"question":question})  

# Display the generated variants
display(result)

['1. How do LLM agents use task decomposition in problem-solving?',
 '2. Benefits of task decomposition for LLM agents in artificial intelligence.',
 '3. Examples of task decomposition strategies for LLM agents.',
 '4. Challenges of implementing task decomposition for LLM agents.',
 '5. Comparison of task decomposition approaches for LLM agents in different AI systems.']

In [ ]:
# Compose the multi-query retrieval pipeline and run it (generate -> retrieve)
simple_retrieval_chain = generate_queries | retriever.map() 

# Retrieve unique docs
docs = simple_retrieval_chain.invoke({"question":question}) 

# Show unique retrieved documents
display(docs)

[[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Challenges in long-term planning and task decomposition: Planning over a lengthy history and effectively exploring the solution space rem

In [ ]:
 # RRF merges multiple ranked result lists into one fused ranking
def reciprocal_rank_fusion(results: list[list], k=60):
    """ 
    Reciprocal_rank_fusion that takes multiple lists of ranked documents and an optional parameter k used in the RRF formula.
    - Ranks are assigned within each list, e.g. rank 0 for 1st document, rank 1 for 2nd document, etc.
    - The fused score for each document is calculated across all lists.
    """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    display(reranked_results)

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

# Select the top k candidate from RRF 
def top_k_docs(docs, k=3): 
    """
    Select the top k document from the RRF results
    """
    # take top-k documents
    top_k_docs = [doc for doc, score in docs[:k]]

    # return list[Document]
    return top_k_docs

retrieval_chain = generate_queries | retriever.map() | reciprocal_rank_fusion | top_k_docs
docs = retrieval_chain.invoke({"question": question})

# Show unique retrieved documents  # present the final selected docs
display(docs)

/var/folders/2n/7qg7n4m55nn6p0lx1w8bgmqh0000gn/T/ipykernel_56383/175969847.py:24: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'),
  0.08042274705581157),
 (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.'),
  0.06558258417063283),
 (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outl

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs

In [7]:
# RAG - build and run final chain that answers the question using retrieved context
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

# Build the prompt object
prompt = ChatPromptTemplate.from_template(template)

# Configure chat LLM (deterministic)
llm = ChatOpenAI(temperature=0) 

# Compose pipeline: retrieval_chain -> prompt -> llm -> parse
final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()  # parse final output to string
)

# Execute the chain and get the answer
final_rag_chain.invoke({"question":question})

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'),
  0.08042274705581157),
 (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.'),
  0.06585580821434867),
 (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outl

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals through methods such as simple prompting, task-specific instructions, or human inputs.'